<a href="https://colab.research.google.com/github/compasstar/Crime-statistics/blob/master/DSC_crime_statistics_ipynb%EC%9D%98_%EC%B5%9C%EC%A2%85%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모듈, 폰트 설치

In [ ]:
# 나눔폰트를 설치하기 (설치 후 Runtime을 재실행 해야 한글을 출력할 수 있습니다.)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy
import math 
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
import random

plt.rc('font', family='NanumBarunGothic') 

# BinarySearchTree Class

In [13]:
# 노드생성 클래스로 노드에는 key(범죄자 고유 분류번호)값과 value(범죄자데이터: [성별, 나이, 종교, 요일, 공범수])
class Node():
    
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.left = None
        self.right = None
        
    def countChildren(self):
        count = 0
        
        if self.left:
            count += 1
        if self.right:
            count += 1
        return count
        
    # 해당 키값을 가진 노드 찾기    
    def searchNode(self, key, parent = None):
        if self.key > key:
            if self.left:
                return self.left.searchNode(key, self)
            else:
                return None, None
        elif self.key < key:
            if self.right:
                return self.right.searchNode(key, self)
            else:
                return None, None
        else:
            return self, parent

    # 새로운 노드 삽입    
    def insertNode(self, key, value):
        if self.key > key:
            if self.left:
                return self.left.insertNode(key, value)
            else:
                self.left = Node(key, value)
                
        elif self.key < key:
            if self.right:
                return self.right.insertNode(key, value)
            else:
                self.right = Node(key, value)
                
        else:
            print("에러")
            
    # 전위 순회하며 모든 value값 반환             
    def preOrderPrintNode(self):
        traversal = []

        traversal.append(self.value)
        if self.left:
            traversal += self.left.preOrderPrintNode()
        if self.right:
            traversal += self.right.preOrderPrintNode()    
            
        return traversal

    # 전위 순회하며 모든 key값 반환
    def preOrderPrintKey(self):
        traversal = []

        traversal.append(self.key)
        if self.left:
            traversal += self.left.preOrderPrintKey()
        if self.right:
            traversal += self.right.preOrderPrintKey()    
            
        return traversal

    # 전위순회하며 특정 조건에 맞는 범죄자 데이터 수 구하기 (traversal의 길이로 특정 조건에 해당하는 데이터 수를 구할 것임)
    # count 
    def preOrderCount(self, inform):
        traversal = []

        if (self.value[0] == inform[0] or inform[0] == '*') and (self.value[1] == inform[1] or inform[1] == '*') and (self.value[2] == inform[2] or inform[2] == '*') and (self.value[3] == inform[3] or inform[3] == '*') and (self.value[4] == inform[4] or inform[4] == '*'):
            traversal.append(0)

        if self.left:
            traversal += self.left.preOrderCount(inform)
        if self.right:
            traversal += self.right.preOrderCount(inform)    
            
        return traversal

    # 전위순회하며 하나의 조건에 맞는 범죄자 데이터 수 구하기 (traversal의 길이로 하나의 조건에 해당되는 데이터 수를 구할 것임)
    # graph 
    def preOrderDataCount(self, index, data):
        traversal = []
        if self.value[index] == data:
          traversal.append(0)

        if self.left:
            traversal += self.left.preOrderDataCount(index, data)
        if self.right:
            traversal += self.right.preOrderDataCount(index, data)    

        return traversal

    # 전위순회하며 각 조건별 리스트에 값 저장 (회귀분석을 하기 위해 값을 문자에서 숫자로 바꾸어준다)
    # 예를 들어 [남, 여, 여, 남, 미상] 순서라면, sex_num = [1, 2, 2, 1, 3]로 저장이 될 것이다
    # correlation
    def preOrderCorrelation(self):
        sex_num = []
        age_num = []
        religion_num = []
        day_num = []
        accom_num = []

        if self.value[0]=='남':
          sex_num.append(1)
        elif self.value[0]=='여':
          sex_num.append(2)
        else:
          sex_num.append(3)

        if self.value[1]=='25세미만':
          age_num.append(1)
        elif self.value[1]=='26_30세':
          age_num.append(2)
        elif self.value[1]=='31_35세':
          age_num.append(3)
        elif self.value[1]=='36_40세':
          age_num.append(4)
        elif self.value[1]=='41_45세':
          age_num.append(5)
        elif self.value[1]=='46_50세':
          age_num.append(6)
        elif self.value[1]=='51_55세':
          age_num.append(7)
        elif self.value[1]=='56_60세':
          age_num.append(8)
        elif self.value[1]=='61_70세':
          age_num.append(9)
        elif self.value[1]=='71세이상':
          age_num.append(10)
        else:
          age_num.append(11)

        if self.value[2]=='불교':
          religion_num.append(1)
        elif self.value[2]=='기독교':
          religion_num.append(2)
        elif self.value[2]=='천주교':
          religion_num.append(3)
        elif self.value[2]=='기타':
          religion_num.append(4)
        elif self.value[2]=='무교':
          religion_num.append(5)
        else:
          religion_num.append(6)

        if self.value[3]=='월':
          day_num.append(1)
        elif self.value[3]=='화':
          day_num.append(2)
        elif self.value[3]=='수':
          day_num.append(3)
        elif self.value[3]=='목':
          day_num.append(4)
        elif self.value[3]=='금':
          day_num.append(5)
        elif self.value[3]=='토':
          day_num.append(6)
        else:
          day_num.append(7)

        if self.value[4] == '단독':
          accom_num.append(1)
        elif self.value[4] == '2명':
          accom_num.append(2)
        elif self.value[4] == '3명 이상':
          accom_num.append(3)
        else:
          accom_num.append(4)

        # 전위 순회하며 각 리스트별로 데이터 추가
        if self.left:
          sex, age, religion, day, accom = self.left.preOrderCorrelation()
          sex_num += sex
          age_num += age
          religion_num += religion
          day_num += day
          accom_num += accom

        if self.right:
          sex, age, religion, day, accom = self.right.preOrderCorrelation()
          sex_num += sex
          age_num += age
          religion_num += religion
          day_num += day
          accom_num += accom

        return sex_num, age_num, religion_num, day_num, accom_num
        

# 트리구조 클래스
class BinarySearchTree():
    
    def __init__(self):
        self.root = None
        
    # 해당 key와 value값을 가진 노드를 삽입한다    
    def insertTree(self, key, value):    
        if self.root:
            self.root.insertNode(key, value)            
        else:
            self.root = Node(key, value)
            
    # 해당 키 값을 가진 노드를 삭제한다        
    def deleteNode(self, key):
        
        # 해당 키 값을 가진 노드 찾기
        node, parent = self.root.searchNode(key)
        if node == None:
            return False
        print(node.key)
        
        countChildren = node.countChildren()
        
        # 1. 해당 노드에게 자식 노드가 없을 경우
        if countChildren == 0:
            
            # 예외처리: 만일 트리에 root 하나밖에 노드가 없을 경우
            if parent == None:
                self.root = None
                
            else:
                if parent.left == node:
                    parent.left = None
                else:
                    parent.right = None
                
        
        # 2. 해당 노드에게 자식 노드가 하나만 있을 경우
        elif countChildren == 1:
            
            if node.left:
                child = node.left
            else:
                child = node.right
                        
            if parent.left == node:
                parent.left = child
            elif parent.right == node:
                parent.right = child
            # node가 root일 경우
            else:
                self.root = child
        
        
        
        # 3. 해당 노드에게 자식 노드가 둘이 있을 경우
        else:
            
            parent = node
            successor = node.right
            
            while successor.left:
                parent = successor
                successor = successor.left
                
            node.key = successor.key
            
            if parent.right == successor:
                parent.right = successor.right
            
            else:
                parent.left = successor.right
        
        return True
        
        
    # 해당 키를 가진 노드를 찾는다
    def searchTree(self, key):
        if self.root:
            return self.root.searchNode(key)
        else:
            print("X")
            return None, None
    
    # 전위순회하며 value값 리턴
    def preOrderPrintTree(self):
        if self.root:
            return self.root.preOrderPrintNode()
        else:
            return []

    # 전위순회하며 key값 리턴
    def preOrderPrintTreeKey(self):
        if self.root:
            return self.root.preOrderPrintKey()
        else:
            return []

    # 전위순회하며 특정 조건 가진 데이터 수 세기
    def preOrderTreeCount(self, value):
      if self.root:
          return self.root.preOrderCount(value)
      else:
          return []

    # 전위순회하며 하나의 조건 가진 데이터 수 세기
    def preOrderTreeDataCount(self, index, data):
        if self.root:
            return self.root.preOrderDataCount(index, data)
        else:
            return []

    # 회귀분석하기 위해 조건 별로 숫자로 바꾸어 리스트화 시켜서 리턴: Node클래스의 preOrderCorrealation() 참고
    def preOrderTreeCorrelation(self):
      if self.root:
        return self.root.preOrderCorrelation()
      else:
        return []

# 범죄자 데이터 형성

In [14]:
# 클래스 생성
criminal = BinarySearchTree()
# 데이터나열
religion = ['불교', '기독교', '천주교', '기타', '무교', '무교', '무교', '무교', '무교', '무교', '무교','무교', '미상', '미상', '미상', '미상', '미상', '미상', '미상', '미상', '미상', '미상', '미상', '미상', '미상', '미상', '미상', '미상']
sex = ['남', '남', '남', '남', '남', '남', '여', '여', '미상']
age = ['25세미만', '26_30세', '31_35세', '36_40세', '41_45세', '46_50세', '51_55세', '56_60세', '61_70세', '71세이상', '미상']
accom = ['단독', '단독', '단독', '단독', '단독', '단독', '단독', '단독', '단독', '단독', '단독', '단독', '단독', '단독', '2명', '2명', '2명', '3명 이상', '기타'] # 공범 수
day = ['월', '화', '수', '목', '금', '토', '일']

# temp에 각 항목의 데이터가 하나씩 저장된 후, criminal로 생성된 트리에 삽입한다
for i in range(100):
  temp = []
  temp.insert(0, random.choice(sex))
  temp.insert(1, random.choice(age))
  temp.insert(2, random.choice(religion))
  temp.insert(3, random.choice(day))
  temp.insert(4, random.choice(accom))
  criminal.insertTree(i+1, temp)

In [ ]:
criminal.preOrderPrintTree()

# CRUD 등 기능 함수

In [16]:

# criminal [성별, 나이, 종교, 요일, 공범수]


# CRUD
# create
def crime_create(key, inform):
  # inform = [성별, 나이, 종교, 요일, 공범수]
  criminal.insertTree(key, inform)


# retrieve
def crime_retrieve(key):
  child, parent = criminal.searchTree(key)
  print(child.value)
  print()

    

# update
def crime_update(key, category, inform):
  tempt = []

  # 어느 정보를 업데이트할 것이냐 정하는 과정
  if category == '성별':
    r = 1
  elif category == '나이':
    r = 2
  elif category == '종교':
    r = 3
  elif category == '요일':
    r = 4
  elif category == '공범수':
    r = 5
  else:
    print("잘못된 데이터 종류를 입력하셨습니다\n")
    return False

  # 해당 키값을 가진 노드 찾기
  child, parent = criminal.searchTree(key)

  # data와 tempt 스택을 만든다. data에서 하나씩 pop하며 tempt에 옮긴 뒤
  # 데이터를 업데이트 하고, 다시 tempt에서 하나씩 pop하며 data에 옮겨준다
  for c in range(6-r):
    data = child.value.pop()
    tempt.append(data)

  # 수정할 데이터는 pop하고, update할 데이터는 append한다
  tempt.pop()
  child.value.append(inform)

  for i in range(len(tempt)):
    data = tempt.pop()
    child.value.append(data)



# delete
def crime_delete(key):
  criminal.deleteNode(key)


# CRUD 외 기능들

# count
def countCriminal(inform):
  count = len(criminal.preOrderTreeCount(inform))

  print("성별:", inform[0], " 나이:", inform[1], " 종교:", inform[2], " 요일:", inform[3], " 공범수:", inform[4], "에 해당되는 범죄자 수는 ", count, "명 입니다")
    

# graph
def graph(category):

  if category == '성별':

    # 각 조건에 맞는 데이터의 수를 세고, 비율로 그래프를 그린다
    # 맨 위의 폰트를 받고 런타임을 재실행해야 한글이 깨지지 않는다
    ratio = []
    ratio.append(len(criminal.preOrderTreeDataCount(0, '남')))
    ratio.append(len(criminal.preOrderTreeDataCount(0, '여')))
    ratio.append(len(criminal.preOrderTreeDataCount(0, '미상')))
    labels = ['남', '여', '미상']

    plt.pie(ratio, labels=labels, autopct='%.1f%%')
    plt.show()

  elif category == '나이':

    ratio = []
    ratio.append(len(criminal.preOrderTreeDataCount(1, '25세미만')))
    ratio.append(len(criminal.preOrderTreeDataCount(1, '26_30세')))
    ratio.append(len(criminal.preOrderTreeDataCount(1, '31_35세')))
    ratio.append(len(criminal.preOrderTreeDataCount(1, '36_40세')))
    ratio.append(len(criminal.preOrderTreeDataCount(1, '41_45세')))
    ratio.append(len(criminal.preOrderTreeDataCount(1, '45_50세')))
    ratio.append(len(criminal.preOrderTreeDataCount(1, '51_55세')))
    ratio.append(len(criminal.preOrderTreeDataCount(1, '56_60세')))
    ratio.append(len(criminal.preOrderTreeDataCount(1, '61_70세')))
    ratio.append(len(criminal.preOrderTreeDataCount(1, '71세이상')))
    ratio.append(len(criminal.preOrderTreeDataCount(1, '미상')))

    labels = ['25세미만', '26_30세','31_35세','36_40세','41_45세','46_50세','51_55세','56_60세','61_70세','71세이상', '미상']

    plt.pie(ratio, labels=labels, autopct='%.1f%%')
    plt.show()



  elif category == '종교':
    
    ratio = []
    ratio.append(len(criminal.preOrderTreeDataCount(2, '불교')))
    ratio.append(len(criminal.preOrderTreeDataCount(2, '기독교')))
    ratio.append(len(criminal.preOrderTreeDataCount(2, '천주교')))
    ratio.append(len(criminal.preOrderTreeDataCount(2, '기타')))
    ratio.append(len(criminal.preOrderTreeDataCount(2, '무교')))
    ratio.append(len(criminal.preOrderTreeDataCount(2, '미상')))

    labels = ['불교', '기독교', '천주교', '기타', '무교', '미상']

    plt.pie(ratio, labels=labels, autopct='%.1f%%')
    plt.show()


  elif category == '요일':

    ratio = []
    ratio.append(len(criminal.preOrderTreeDataCount(3, '월')))
    ratio.append(len(criminal.preOrderTreeDataCount(3, '화')))
    ratio.append(len(criminal.preOrderTreeDataCount(3, '수')))
    ratio.append(len(criminal.preOrderTreeDataCount(3, '목')))
    ratio.append(len(criminal.preOrderTreeDataCount(3, '금')))
    ratio.append(len(criminal.preOrderTreeDataCount(3, '토')))
    ratio.append(len(criminal.preOrderTreeDataCount(3, '일')))

    labels = ['월', '화', '수', '목', '금', '토', '일']

    plt.pie(ratio, labels=labels, autopct='%.1f%%')
    plt.show()

  elif category == '공범수':

    ratio = []
    ratio.append(len(criminal.preOrderTreeDataCount(4, '단독')))
    ratio.append(len(criminal.preOrderTreeDataCount(4, '2명')))
    ratio.append(len(criminal.preOrderTreeDataCount(4, '3명 이상')))
    ratio.append(len(criminal.preOrderTreeDataCount(4, '기타')))

    labels = ['단독', '2명', '3명 이상', '기타']

    plt.pie(ratio, labels=labels, autopct='%.1f%%')
    plt.show()

  else:
    print("잘못된 데이터 종류를 입력하셨습니다\n")
    return False



# correlation
def correlation(one, two):

  # 각 조건 별 데이터를 숫자로 바꾸어 준다
  num_list = criminal.preOrderTreeCorrelation()

  # 회귀분석을 하기 위해 데이터프레임화 시켜준다
  df = pd.DataFrame({'성별': num_list[0], '나이': num_list[1], '종교':num_list[2], '요일':num_list[3], '공범수':num_list[4]})

  # 회귀분석모듈 사용
  slope, intercept, rvalue, pvalue, stderr = scipy.stats.linregress(x=df[one], y=df[two])
  
  # pvalue에 따른 상관관계 분석
  if pvalue>0.05:
    print("p-value = %.4f 상관관계가 부족합니다\n" % pvalue)
  else:
    print("p-value = %.4f 해당 데이터를 기준으로 하면 상관이 있을 수 있습니다\n" % pvalue)


  

# 키값 유효성 검사

In [17]:
# 키값 유효성 검사
# 해당 키를 가진 범죄자가 있다면 True 없다면 False를 반환한다
def ok_key(key):

  try:
    key = int(key)
  except:
    return False

  child, parent = criminal.searchTree(key)

  if child != None:
    return True
  else:
    return False

# 프로그램 실행함수

In [18]:
while True:
  print("\n*****************************************************")
  print("범죄자 데이터 관리 프로그램에 오신 것을 환영합니다")
  print("<사용하고자 하는 기능을 선택하세요>")
  print("1. Create\n2. Retreive\n3. Updata\n4. Delete\n5. Count\n6. Graph\n7. Correlation\n8. Exit")
  print("*****************************************************")

  # 해당 값에 따라 기능 선택
  cmd = input("Enter the number ")

  # 잘못된 값을 입력할 시, 예외처리
  try:
    cmd = int(cmd)
  except:
    print("올바른 숫자를 입력하세요\n")
    continue

  # 8번. 프로그램 종료
  if cmd == 8:
    print("프로그램을 종료합니다")
    break

  # 1번. create
  if cmd == 1:
    while True:
      key = input("추가하고자 하는 범죄자 데이터의 key값을 입력하세요 [취소: -1]")

      # 취소시 끝낸다
      if key == '-1':
        break

      # 키값 유효성 검사
      if ok_key(key):
        print("해당 key값을 가진 범죄자 데이터가 이미 존재합니다")
        continue

      inform = list(input("추가하고자 하는 범죄자 데이터를 [성별, 나이, 종교, 요일, 공범수] 순서로 입력하세요\n(예시: 남 26_30세 천주교 목 단독)").split())
      print()

      # 데이터를 올바르게 입력하지 않고, 빼먹거나 할 시 예외처리
      if len(inform) != 5:
        print("올바른 데이터를 입력해주세요\n")
        continue

      # create 실행
      else:
        crime_create(int(key), inform)
        break


  # 2번. retrieve
  elif cmd == 2:
    while True:
      key = input("불러오고자 하는 범죄자의 key값을 입력하세요(ALL이라고 적으면 전부 출력합니다) [취소: -1]\n(예시: 50) ")

      # 취소시 끝낸다
      if key == '-1':
        break

      # ALL을 입력했을 경우
      if key == 'ALL':
        key = criminal.preOrderPrintTreeKey()
        value = criminal.preOrderPrintTree()

        for pair in zip(key, value):
          print(pair)
        break

      # 키값 유효성 검사
      if ok_key(key) == False:
        print("해당 key값을 가진 범죄자 데이터는 존재하지 않습니다\n")
        continue

      # retrieve 실행
      crime_retrieve(int(key))
      break


  # 3번. update
  elif cmd == 3:
    while True:
      L = list(input("업데이트하고자 하는 범죄자의 key값과, 데이터의 종류, 수정된 데이터를 차례로 입력하세요 [취소: -1]\n(예시: 2 종교 천주교) ").split())
      
      # 취소시 종료
      if L[0] == '-1':
        break

      try:
        crime_update(int(L[0]), L[1], L[2])
        break
      # 올바른 데이터를 입력하지 않을 시 예외처리
      except:
        print("올바른 데이터를 입력하세요\n")
        continue


  
    
  # 4번. delete
  elif cmd == 4:
    while True:
      key = input("지우고자 하는 범죄자의 key값을 입력하세요 [취소: -1]\n(예시: 50) ")

      # 취소 시 끝낸다
      if key == '-1':
        break

      # 인덱스 유효성 검사
      if ok_key(key):
        # delete 실행
        crime_delete(int(key))
        break
      else:
        continue




  # 5번. count
  elif cmd == 5:
    while True:
      inform = list(input("세고자 하는 범죄자 데이터를 입력하세요 ([성별, 나이, 종교, 요일, 공범수] 상관없다면 *를 대신 입력하시면 됩니다) [취소: -1]\n(예시: 남 * 기독교 금 *) ").split())
      print()

      # 취소 시 끝낸다
      if inform[0] == '-1':
        break

      try:
        # count 실행
        countCriminal(inform)
        break
      except:
        print("올바른 범죄자 데이터를 입력하세요.\n")
        continue

  # 6번. graph
  elif cmd == 6:
    while True:
      category = input("그래프를 그리고자 하는 범죄자 데이터 종류를 [성별, 나이, 종교, 요일, 공범수] 중에 입력하세요 [취소: -1]\n(예시: 요일) ")
      
      # 취소시 종료
      if category == '-1':
        break

      try:
        # 그래프 실행
        graph(category)
        break
      # 올바른 데이터 종류를 입력하지 않을 시 예외처리
      except:
        print("올바른 데이터 종류를 입력하세요")
        continue

  # 7번. correlation
  elif cmd == 7:
    while True:
      L = list(input("상관도를 분석하고자 하는 데이터를 [성별, 나이, 종교, 요일, 공범수] 중에 두개 선택하세요 [취소: -1]\n(예시: 성별 나이) ").split())
      
      # 취소시 종료
      if L[0] == '-1':
        break

      try:
        # correlation 실행
        correlation(L[0], L[1])
        break
      except:
        print("올바른 데이터를 입력하세요\n")
  
  # 1번~8번 외를 입력했을 시
  else:
    print("올바른 숫자를 입력하세요")
    continue


*****************************************************
범죄자 데이터 관리 프로그램에 오신 것을 환영합니다
<사용하고자 하는 기능을 선택하세요>
1. Create
2. Retreive
3. Updata
4. Delete
5. Count
6. Graph
7. Correlation
8. Exit
*****************************************************
Enter the number 2
불러오고자 하는 범죄자의 key값을 입력하세요(ALL이라고 적으면 전부 출력합니다) [취소: -1]
(예시: 50) 100
['여', '36_40세', '미상', '목', '단독']


*****************************************************
범죄자 데이터 관리 프로그램에 오신 것을 환영합니다
<사용하고자 하는 기능을 선택하세요>
1. Create
2. Retreive
3. Updata
4. Delete
5. Count
6. Graph
7. Correlation
8. Exit
*****************************************************
Enter the number 8
프로그램을 종료합니다
